<a href="https://colab.research.google.com/github/EthanCui2008/Kaggle/blob/Main/Arxiv_Webscraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [12]:
!pip install python-dotenv

In [50]:
!pip install pylatexenc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=862d1353ef0d6797a64efd97643da23da57f77ef4a796b8a9dee38c72affbf13
  Stored in directory: /root/.cache/pip/wheels/d3/31/8b/e09b0386afd80cfc556c00408c9aeea5c35c4d484a9c762fd5
Successfully built pylatexenc


In [82]:
!pip install beautifulsoup4

In [84]:
import requests
import tarfile
import io
import re
import os
import json

In [57]:
from pylatexenc.latexwalker import LatexWalker, LatexGroupNode, LatexMacroNode, LatexEnvironmentNode

In [17]:
from dotenv import load_dotenv
load_dotenv('Key.env')

True

In [88]:
from openai import OpenAI

In [9]:
from google.colab import drive
directory_path = '/content/drive/MyDrive/Arxiv_Project[Temp]'
os.chdir(directory_path)

In [98]:
#Make this a repeatable call to scrape Arxiv

url = 'https://arxiv.org/pdf/1706.03762'
response = requests.get(url)

In [85]:
functions = [
    {
        "name": "extract_researcher_info",
        "description": "Extracts researcher names and their organizations from a given text.",
        "parameters": {
            "type": "object",
            "properties": {
                "text": {
                    "type": "string",
                    "description": "The text containing researcher names and their organizations."
                },
            },
            "required": ["text"],
        },
    }
]

In [99]:
#Used to extract relevant parts of a Latex file to be function called

def extract_block(latex_text, block_name):
    #Initializes the walker for Latex
    walker = LatexWalker(latex_text)
    nodelist, pos, length = walker.get_latex_nodes()

    block = None

    #Loops through nodelist
    for node in nodelist:
        #Checks if latex node is a macro_node[big node], and is also the desired blockname
        if isinstance(node, LatexMacroNode) and node.macroname == block_name:
            #Capture all content in latex node block
            block = ''.join(
                arg.latex_verbatim() for arg in node.nodeargd.argnlist
            )
            break
    return block

In [79]:
#Extracts tar.gz file from arxiv to anaylize

#r:gz unzips as a tar.gz file
#fileobj=io.BytesIO means it accepts the data from a webstream, meaning it does need to be written to disk
with tarfile.open(fileobj=io.BytesIO(response.content), mode='r:gz') as tar:

    #replace with function that finds the main .tex file
    tar_info = tar.getmember('ms.tex')

    #extracts tar file, then reads as text -> feed into
    with tar.extractfile(tar_info) as file:
        content = file.read().decode('utf-8')

In [83]:
author_block = extract_block(content, 'author')

In [97]:
print(author_block)

{
  \AND
  Ashish Vaswani\thanks{Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started the effort to evaluate this idea.
Ashish, with Illia, designed and implemented the first Transformer models and has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head attention and the parameter-free position representation and became the other person involved in nearly every detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating our research.
}\\
  Google Brain\\
  \texttt{avaswani@google.com}\\
  \And
 

In [93]:

# API call to GPT-4 with function calling

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("API_KEY"),
)
response = client.chat.completions.create(
    model="gpt-4-0613",  # Model that supports function calling
    messages=[
        {"role": "system", "content": "You are an AI that extracts names and their organizations."},
        {"role": "user", "content": f"Extract researcher names and their organizations from the following text: {author_block}"},
    ],
    functions=functions,
    function_call={"name": "extract_researcher_info"},  # Force the function call
)


[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', function_call=FunctionCall(arguments='{\n  "text": "Ashish Vaswani\\\\nGoogle Brain\\\\navaswani@google.com\\\\Noam Shazeer\\\\nGoogle Brain\\\\nnoam@google.com\\\\Niki Parmar\\\\nGoogle Research\\\\nnikip@google.com\\\\Jakob Uszkoreit\\\\nGoogle Research\\\\nusz@google.com\\\\Llion Jones\\\\nGoogle Research\\\\nllion@google.com\\\\Aidan N. Gomez\\\\nUniversity of Toronto\\\\naidan@cs.toronto.edu\\\\{\\L}ukasz Kaiser\\\\nGoogle Brain\\\\nlukaszkaiser@google.com\\\\Illia Polosukhin\\\\nillia.polosukhin@gmail.com"\n}', name='extract_researcher_info'), tool_calls=None))]


In [96]:
print(response.function_call.arguments)

AttributeError: 'ChatCompletion' object has no attribute 'function_call'